In [1]:
!nvidia-smi

Tue May 17 11:55:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install -U synonyms

     |████████████████████████████████| 10.7 MB 5.4 MB/s 
  Created wheel for synonyms: filename=synonyms-3.16.0-py3-none-any.whl size=10832785 sha256=220470d422748f130d6722c5a1494d2fc890785c3b511a06309c38fee51af9c9
  Stored in directory: /root/.cache/pip/wheels/e4/cd/43/b4548753509a94471fc946967a07116252d49aeeb689db8f7c
Successfully built synonyms


In [4]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [5]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 2.9 MB 31.8 MB/s 
     |████████████████████████████████| 880 kB 39.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f7e1238aa9a9a56c8b9f5d87f0a1bff5c4fae79d0c154fab2f2c948be6394acd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/usr/local/lib/python3.7/dist-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/usr/local/lib/python3.7/dist-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz] ...

[Synonyms] downloading data from https://github.com/chatopera/Synonyms/releases/download/3.15.0/words.vector.gz to /usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz ... 
 this only happens if SYNONYMS_WORD2VEC_BIN_URL_ZH_CN is not present and Synonyms initialization for the first time. 
 It would take minutes that depends on network.

[Synonyms] downloaded.



## EDA（Easy Data Augmentation）
![EDA3](https://img-blog.csdnimg.cn/50c22b4212714b509ce053ff921d6bdd.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

In [7]:
# 读取停用词表
import random
import re
from random import shuffle
# strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
stop_words = {word.strip() for word in open('/content/drive/MyDrive/Colab Notebooks/dataset/baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [8]:
# 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。

# 得到同义词列表
def get_synonym(word):
  # nearby返回一个元组，位置0返回同义词，位置1返回相似度
  sys = set(synonyms.nearby(word)[0])
  # 去除原词
  if word in sys:
    sys.remove(word)
  return list(sys)
  # 如果输入"给力" 可能没有同义词（同义词只有他自己） 则返回  ([],[])

# 同义词替换
# 传入一个词列表和替换词的数量
def synonym_replacement(words, n):

  new_words = words.copy()
  # 去除停用词，去重，变成列表
  random_word_list = list(set([word for word in words if word not in stop_words]))
  # 打乱
  random.shuffle(random_word_list)

  num_replaced = 0
  for random_word in random_word_list:
    synonym_words = get_synonym(random_word)
    if len(synonym_words)>=1:
        # 随机选取一个同义词
        synonym = random.choice(list(synonym_words))
        # 如果word是要替换的词 替换成同义词 ，否则返回原词
        new_words = [synonym if word == random_word else word for word in new_words]
        # 同义词替换数量+1
        num_replaced += 1
        
    if num_replaced >= n:
        break
  # 为什么加这两句话？ 看note1
  sentence = ' '.join(new_words)
  new_words = sentence.split(' ')

  return new_words

In [9]:
# 使用eda进行数据增强
# 同义词替换的比例、 增强的句子数目
def eda(sentence, alpha_sr=0.1, num_aug=9):

  # 位置0是词组、1是词性
  words = synonyms.seg(sentence)[0]
  num_words = len(words)
  n_sr = max(1, int(alpha_sr * num_words))

  augmented_sentences = []

  for _ in range(num_aug):
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences.append(' '.join(a_words))
  return augmented_sentences

In [10]:
# note1
# 存在这样一种情况
sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'actor']
word = 'actor'
example_synonyms = ['actress', 'film star', 'performer', 'comedian', 'entertainer']
new_sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'film star']
# 为了消除空格
new_sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'film', 'star']

In [11]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

['9 月 15 日 以来 ， 三星电子 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 货源 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 中小企业 ， 也 因 采用 美国 技术 ， 而 无法 购进 给 华为 。 目前 华为 部分 型号 的 笔记本 产品 出现 白米 少 的 现象 ， 若 该 金融形势 持续 下去 ， 华为 笔记本 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， GT5316SB0 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 甚至 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 选用 美国 技术 ， 而 无法 供货商 给 华为 。 目前 华为 部分 车种 的 笔记本 产品 出现 货 少 的 现象 ， 若 该 形势 在短期内 下去 ， 华为 笔记本 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 西部数据 、 三星 等 华为 的 重要 SAP ， 只要 没有 加拿大 的 相关 备案 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 加拿大 半导体技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 机种 的 手机 产品 出现 猪仔 少 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 本月 15 日晚 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 器件 给 华为 ， 而 杜博韦 等 国产 器件 企业 ， 也 因 装配 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 该 政治形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 年末 15 日 以来 ， Tektronix 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 新西兰 的 上述 许可证 ， 都 无法 供货 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 新西兰 控制技术 ， 而 无法 供货 给 华为 。 目前 华为 部